In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/ruyogagp/medical_interpretability')

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from cga import cga
import pandas as pd
import neptune.new as neptune
import wandb
import pytorch_lightning as pl
import torch.nn as nn
import torch
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from captum.attr import *
from captum.metrics import *
from captum._utils.models.linear_model import SkLearnLinearRegression
import shap_fork as shap
from source.tasks import DeepSurv
from source.wrappers import ForwardWrapper

In [3]:
results_directory = '/data/analysis/ag-reils/ag-reils-shared/cardioRS/results/interpretability/correlation_case'
data_directory = '/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/interpretability/correlation'

## Helper Functions

In [4]:
def load_details(name, project='simpsons'):
    dirpath = '/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/interpretability/simpson' if project=='simpsons' else '/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/interpretability/correlation'
    train = pd.read_csv(f'{dirpath}/{name}_train_details.csv')
    valid = pd.read_csv(f'{dirpath}/{name}_valid_details.csv')
    return train, valid

def df2csv(
        df: pd.DataFrame,
        name: str,
        output_dir: str,
):
    """
    Writes csv given a dataframe + name
    """
    train, valid = train_test_split(df, test_size=0.3)
    train.to_csv(
        f"{output_dir}/{name}_train_details.csv",
        index=False,
    )
    valid.to_csv(
        f"{output_dir}/{name}_valid_details.csv",
        index=False,
    )

    train_df = train.loc[:, ['x_orig', 'y_orig', 'time', 'event']]
    valid_df = valid.loc[:, ['x_orig', 'y_orig', 'time', 'event']]
    train_df.to_csv(
        f"{output_dir}/{name}_train.csv",
        index=False,
    )
    valid_df.to_csv(
        f"{output_dir}/{name}_valid.csv",
        index=False,
    )

## Load from wandb

In [14]:
experiment_id = 'p-0.73'
attribute_details = pd.read_csv(f'{data_directory}/{experiment_id}_attribute_details.csv')
attribution_names = ["FeaturePermutation",
                     "FeatureAblation",
                     "KernelExplainer",
                     "DeepExplainer",
                     "ShapleyValueSampling",
                     "IntegratedGradients",
                     "InputxGradient",
                     "Lime",
                     "Saliency"]

# get valid runs
api = wandb.Api()
runs =  api.runs('cardiors/interpretability',
                 filters={"$and": [{'tags': f'{experiment_id}'}, {'state': 'finished'}]})

In [12]:
fa_paths = []
#TODO: this is hardcoded
for run in runs:
    try:
        # TODO: this is hardcoded
        fa_paths.append(run.config['FeatureAblation_path'])
    except:
        print(run.experiment_id)

In [13]:
fa_path = fa_paths[0]
attribution = np.genfromtxt(fa_path, delimiter=',')

## Load details

In [15]:
details_df = attribute_details

In [ ]:
def change_slope(details_df)